In [ ]:
import os
import urllib.request
import librosa
import soundfile
import numpy as np
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename
import pickle
import pandas as pd

with open(f'model/modelv9.pkl', 'rb') as f:
    model = pickle.load(f)

UPLOAD_FOLDER = './uploads'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['wav'])


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('upload.html', intro=True)

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('Invalid File')
            return render_template('upload.html', intro=False)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            flash('File successfully uploaded')
            feat = extract_feature(myfile)
            feat = feat.reshape(1,-1)
            prediction = model.predict(feat)
            prediction = prediction[0].split('_')
            
            return  render_template('upload.html', result=True, gender=prediction[1], mood=prediction[0])
        else:
            flash('Allowed file type is just wav for now')
            return render_template('upload.html', intro=False)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2020 00:48:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 00:48:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 00:48:17] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [11/Apr/2020 00:48:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 00:48:17] "GET / HTTP/1.1" 200 -
/Applications/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
127.0.0.1 - - [11/Apr/2020 00:48:34] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 00:48:34] "GET / HTTP/1.1" 200 -
